In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

import os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
gpt4o = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    api_key=os.getenv("API_KEY")
)

In [3]:
# Create the primary agent.
fstAgent = AssistantAgent(
    "primary",
    model_client=gpt4o,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
sndAgent = AssistantAgent(
    "critic",
    model_client=gpt4o,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)
# Define a termination condition that stops the task if the critic approves.
approved = TextMentionTermination("APPROVE")

In [4]:
# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([fstAgent, sndAgent], termination_condition=approved)

In [12]:
# Now run the team til the end
# result = await team.run(task="Write a short poem about the fall season.")
# print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=118), content="Leaves of amber gently fall,  \nWhispering the end of summer's call.  \nA crisp wind dances through the trees,  \nAs nature’s palette blazes with ease.  \n\nGolden hues and scarlet dreams,  \nMirror the sun’s soft, dwindling beams.  \nPumpkin patches and fireside eves,  \nEcho the harvest's fruitful reprieves.  \n\nThe world slows in autumn's embrace,  \nTime to reflect, to pause, to grace.  \nIn the quiet rustle of fall's gentle gust,  \nWe find beauty in change and learn to trust.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=163, completion_tokens=145), content='This poem beautifully captures the essence of the fall season. The imagery of "leaves of amber" and "nature\'s palette blazes with eas

In [5]:
# Run the team and wait for incoming messages.
async for message in team.run_stream(task="Write a short poem about the fall season, in french, in 4 sentences."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season, in french, in 4 sentences.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=36, completion_tokens=44) content="Les feuilles dansent au vent léger,  \nCouleurs d'or et de rouille dispersées.  \nLe parfum des pommes embaume les champs,  \nL'automne chante sa douce mélodie des temps." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=97, completion_tokens=238) content='Your poem beautifully captures the essence of the fall season with vivid imagery and sensory details. The use of personification and the evocation of smell and sound add depth to the experience of autumn. Here are a few suggestions to further refine it:\n\n1. In the first line, consider emphasizing the gracefulness of the leaves\' movement to enhance the visual imagery. For example: "Les feuilles dansent avec grâce au vent léger".\n\n2. The second line could benefit from a bit more specific

In [6]:
# Ou plus simplement
await team.reset()
Console(team.run_stream(task="Write a short poem about the fall season, in french, in 4 sentences."))

<coroutine object Console at 0x753405d74640>